In [1]:
import sys
sys.path.append("../../pybdp")
from src import pybdp
#import pybdp
from pprint import pprint

# Start with an empty project
project = pybdp.create_empty_project()

project.add_space(id = "X",
                  name = "X",
                  description = "Input vector")

project.add_space(id = "x_i",
                  name = "x_i",
                  description = "Individual token")

project.add_space(id = "h_i",
                  name = "h_i",
                  description = "Hidden state")

project.add_space(id = "c",
                  name = "c",
                  description = "Context vector")

project.add_block(id="RNN Encoder Hidden Layer",
                  name="RNN Encoder Hidden Layer",
                  description="A hidden encoder layer of an RNN",
                  domain=["x_i", "h_i"],
                  codomain=["h_i"],)

project.add_block(id="Input Layer Block Length 3",
                  name="Input Layer Block Length 3",
                  description="The input layer which transforms the input vector into a sequence of tokens",
                  domain=["X"],
                  codomain=["x_i", "x_i", "x_i"],)

project.add_processor(id="Input Layer Length 3",
                  description="The input layer which transforms the input vector into a sequence of tokens",
                  parent_id="Input Layer Block Length 3",)


project.add_block(id="Build Context Vector Block Length 3",
                  name="Build Context Vector Block Length 3",
                  description="The block which builds the context vector",
                  domain=["h_i", "h_i", "h_i"],
                  codomain=["c"],)

project.add_processor(id="Build Context Vector Length 3",
                  description="The block which builds the context vector",
                  parent_id="Build Context Vector Block Length 3",)


project.add_processor(id="RNN Encoder Cell 1", parent_id="RNN Encoder Hidden Layer")
project.add_processor(id="RNN Encoder Cell 2", parent_id="RNN Encoder Hidden Layer")
project.add_processor(id="RNN Encoder Cell 3", parent_id="RNN Encoder Hidden Layer")

In [2]:
pprint(project.processors_map["RNN Encoder Cell 1"].find_potential_wires(project.processors_map["RNN Encoder Cell 2"]))
pprint(project.processors_map["RNN Encoder Cell 2"].find_potential_wires(project.processors_map["RNN Encoder Cell 3"]))

{'Ports': [{'Parent': 'h_i',
            'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 2'},
            'Target': {'Index': 1, 'Processor': 'RNN Encoder Cell 1'}}],
 'Terminals': [{'Parent': 'h_i',
                'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 1'},
                'Target': {'Index': 1, 'Processor': 'RNN Encoder Cell 2'}}]}
{'Ports': [{'Parent': 'h_i',
            'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 3'},
            'Target': {'Index': 1, 'Processor': 'RNN Encoder Cell 2'}}],
 'Terminals': [{'Parent': 'h_i',
                'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 2'},
                'Target': {'Index': 1, 'Processor': 'RNN Encoder Cell 3'}}]}


In [3]:
project.add_wires([{'Parent': 'h_i',
                'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 1'},
                'Target': {'Index': 1, 'Processor': 'RNN Encoder Cell 2'}},
                {'Parent': 'h_i',
                'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 2'},
                'Target': {'Index': 1, 'Processor': 'RNN Encoder Cell 3'}}], auto_increment=True)


project.add_wires([{'Parent': 'x_i',
                'Source': {'Index': 0, 'Processor': 'Input Layer Length 3'},
                'Target': {'Index': 0, 'Processor': 'RNN Encoder Cell 1'}},
                {'Parent': 'x_i',
                'Source': {'Index': 1, 'Processor': 'Input Layer Length 3'},
                'Target': {'Index': 0, 'Processor': 'RNN Encoder Cell 2'}},
                {'Parent': 'x_i',
                'Source': {'Index': 2, 'Processor': 'Input Layer Length 3'},
                'Target': {'Index': 0, 'Processor': 'RNN Encoder Cell 3'}}], auto_increment=True)


project.add_wires([{'Parent': 'h_i',
                'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 1'},
                'Target': {'Index': 0, 'Processor': "Build Context Vector Length 3"}},
                {'Parent': 'h_i',
                'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 2'},
                'Target': {'Index': 1, 'Processor': "Build Context Vector Length 3"}},
                {'Parent': 'h_i',
                'Source': {'Index': 0, 'Processor': 'RNN Encoder Cell 3'},
                'Target': {'Index': 2, 'Processor': "Build Context Vector Length 3"}}], auto_increment=True)

In [4]:
project.add_system(id="RNN Encoder System Length 3",
                   processors=['RNN Encoder Cell 1', 'RNN Encoder Cell 2', 'RNN Encoder Cell 3', 'Input Layer Length 3', "Build Context Vector Length 3"],
                   wires=["W1", "W2", "W3", "W4", "W5", "W6", "W7", "W8"],
                     description="The RNN encoder system",)

project.systems_map["RNN Encoder System Length 3"].display_mermaid_graphic()

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GS0[RNN Encoder System Length 3]
subgraph G0[RNN Encoder Cell 1 - RNN Encoder Hidden Layer Block]
direction LR
X0[RNN Encoder Cell 1]
subgraph G0P[Ports]
direction TB
XX0P0[x_i]
XX0P1[h_i]
end
XX0P0[x_i] o--o X0
XX0P1[h_i] o--o X0
subgraph G0T[Terminals]
direction TB
XX0T0[h_i]
end
X0 o--o XX0T0[h_i]
end
subgraph G1[RNN Encoder Cell 2 - RNN Encoder Hidden Layer Block]
direction LR
X1[RNN Encoder Cell 2]
subgraph G1P[Ports]
direction TB
XX1P0[x_i]
XX1P1[h_i]
end
XX1P0[x_i] o--o X1
XX1P1[h_i] o--o X1
subgraph G1T[Terminals]
direction TB
XX1T0[h_i]
end
X1 o--o XX1T0[h_i]
end
subgraph G2[RNN Encoder Cell 3 - RNN Encoder Hidden Layer Block]
direction LR
X2[RNN Encoder Cell 3]
subgraph G2P[Ports]
direction TB
XX2P0[x_i]
XX2P1[h_i]
end
XX2P0[x_i] o--o X2
XX2P1[h_i] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[h_i]
end
X2 o--o XX2T0[h_i]
end
subgraph G3[Input Layer Length 3 - Input Layer Block Length 3 Block]
direction LR
X3[Input Layer Length 3]
subgraph G3P[Ports]
direction TB
XX3P0[X]
end
XX3P0[X] o--o X3
subgraph G3T[Terminals]
direction TB
XX3T0[x_i]
XX3T1[x_i]
XX3T2[x_i]
end
X3 o--o XX3T0[x_i]
X3 o--o XX3T1[x_i]
X3 o--o XX3T2[x_i]
end
subgraph G4[Build Context Vector Length 3 - Build Context Vector Block Length 3 Block]
direction LR
X4[Build Context Vector Length 3]
subgraph G4P[Ports]
direction TB
XX4P0[h_i]
XX4P1[h_i]
XX4P2[h_i]
end
XX4P0[h_i] o--o X4
XX4P1[h_i] o--o X4
XX4P2[h_i] o--o X4
subgraph G4T[Terminals]
direction TB
XX4T0[c]
end
X4 o--o XX4T0[c]
end
XX0T0[h_i] ---> XX1P1[h_i]
XX1T0[h_i] ---> XX2P1[h_i]
XX3T0[x_i] ---> XX0P0[x_i]
XX3T1[x_i] ---> XX1P0[x_i]
XX3T2[x_i] ---> XX2P0[x_i]
XX0T0[h_i] ---> XX4P0[h_i]
XX1T0[h_i] ---> XX4P1[h_i]
XX2T0[h_i] ---> XX4P2[h_i]
end

```

In [5]:
project.systems_map["RNN Encoder System Length 3"].make_processor_lazy(block_id="RNN Encoder Block Length 3",
                                                                    block_name="RNN Encoder Block Length 3",
                                                                    processor_id="RNN Encoder Length 3",
                                                                    processor_name="RNN Encoder Length 3",
                                                                    block_description="An RNN Encoder Block with length 3",
                                                                    processor_description="An RNN Encoder with length 3",
                                                                    add=True,
                                                                    project=project,)
project.processors_map["RNN Encoder Length 3"].display_mermaid_graphic(composite=True)

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GC0[RNN Encoder Length 3 - RNN Encoder Block Length 3 Block]
direction LR
subgraph GS0[RNN Encoder System Length 3]
subgraph G1[RNN Encoder Cell 1 - RNN Encoder Hidden Layer Block]
direction LR
X1[RNN Encoder Cell 1]
subgraph G1P[Ports]
direction TB
XX1P0[x_i]
XX1P1[h_i]
end
XX1P0[x_i] o--o X1
XX1P1[h_i] o--o X1
subgraph G1T[Terminals]
direction TB
XX1T0[h_i]
end
X1 o--o XX1T0[h_i]
end
subgraph G2[RNN Encoder Cell 2 - RNN Encoder Hidden Layer Block]
direction LR
X2[RNN Encoder Cell 2]
subgraph G2P[Ports]
direction TB
XX2P0[x_i]
XX2P1[h_i]
end
XX2P0[x_i] o--o X2
XX2P1[h_i] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[h_i]
end
X2 o--o XX2T0[h_i]
end
subgraph G3[RNN Encoder Cell 3 - RNN Encoder Hidden Layer Block]
direction LR
X3[RNN Encoder Cell 3]
subgraph G3P[Ports]
direction TB
XX3P0[x_i]
XX3P1[h_i]
end
XX3P0[x_i] o--o X3
XX3P1[h_i] o--o X3
subgraph G3T[Terminals]
direction TB
XX3T0[h_i]
end
X3 o--o XX3T0[h_i]
end
subgraph G4[Input Layer Length 3 - Input Layer Block Length 3 Block]
direction LR
X4[Input Layer Length 3]
subgraph G4P[Ports]
direction TB
XX4P0[X]
end
XX4P0[X] o--o X4
subgraph G4T[Terminals]
direction TB
XX4T0[x_i]
XX4T1[x_i]
XX4T2[x_i]
end
X4 o--o XX4T0[x_i]
X4 o--o XX4T1[x_i]
X4 o--o XX4T2[x_i]
end
subgraph G5[Build Context Vector Length 3 - Build Context Vector Block Length 3 Block]
direction LR
X5[Build Context Vector Length 3]
subgraph G5P[Ports]
direction TB
XX5P0[h_i]
XX5P1[h_i]
XX5P2[h_i]
end
XX5P0[h_i] o--o X5
XX5P1[h_i] o--o X5
XX5P2[h_i] o--o X5
subgraph G5T[Terminals]
direction TB
XX5T0[c]
end
X5 o--o XX5T0[c]
end
XX1T0[h_i] ---> XX2P1[h_i]
XX2T0[h_i] ---> XX3P1[h_i]
XX4T0[x_i] ---> XX1P0[x_i]
XX4T1[x_i] ---> XX2P0[x_i]
XX4T2[x_i] ---> XX3P0[x_i]
XX1T0[h_i] ---> XX5P0[h_i]
XX2T0[h_i] ---> XX5P1[h_i]
XX3T0[h_i] ---> XX5P2[h_i]
end
subgraph GC0P[Ports]
direction TB
X1P0[h_i]
X1P1[X]
end
X1P0[h_i] --> XX1P1[h_i]
X1P1[X] --> XX4P0[X]
subgraph GC0T[Terminals]
direction TB
X1T0[c]
end
XX5T0[c] --> X1T0[c]
end

```

In [6]:
project.add_space(id = "Y",
                  name = "Y",
                  description = "Output vector")

project.add_space(id = "y_i",
                  name = "y_i",
                  description = "Individual output token")

project.add_block(id="RNN Decoder Hidden Layer",
                  name="RNN Decoder Hidden Layer",
                  description="A hidden decoder layer of an RNN",
                  domain=["y_i", "h_i", "c"],
                  codomain=["y_i", "h_i"],)

project.add_processor(id="RNN Decoder Cell 1", parent_id="RNN Decoder Hidden Layer")
project.add_processor(id="RNN Decoder Cell 2", parent_id="RNN Decoder Hidden Layer")
project.add_processor(id="RNN Decoder Cell 3", parent_id="RNN Decoder Hidden Layer")



In [7]:
pprint(project.processors_map["RNN Decoder Cell 1"].find_potential_wires(project.processors_map["RNN Decoder Cell 2"]))
pprint(project.processors_map["RNN Decoder Cell 2"].find_potential_wires(project.processors_map["RNN Decoder Cell 3"]))

{'Ports': [{'Parent': 'y_i',
            'Source': {'Index': 0, 'Processor': 'RNN Decoder Cell 2'},
            'Target': {'Index': 0, 'Processor': 'RNN Decoder Cell 1'}},
           {'Parent': 'h_i',
            'Source': {'Index': 1, 'Processor': 'RNN Decoder Cell 2'},
            'Target': {'Index': 1, 'Processor': 'RNN Decoder Cell 1'}}],
 'Terminals': [{'Parent': 'y_i',
                'Source': {'Index': 0, 'Processor': 'RNN Decoder Cell 1'},
                'Target': {'Index': 0, 'Processor': 'RNN Decoder Cell 2'}},
               {'Parent': 'h_i',
                'Source': {'Index': 1, 'Processor': 'RNN Decoder Cell 1'},
                'Target': {'Index': 1, 'Processor': 'RNN Decoder Cell 2'}}]}
{'Ports': [{'Parent': 'y_i',
            'Source': {'Index': 0, 'Processor': 'RNN Decoder Cell 3'},
            'Target': {'Index': 0, 'Processor': 'RNN Decoder Cell 2'}},
           {'Parent': 'h_i',
            'Source': {'Index': 1, 'Processor': 'RNN Decoder Cell 3'},
            'T

In [8]:
project.add_wires([{'Parent': 'y_i',
                'Source': {'Index': 0, 'Processor': 'RNN Decoder Cell 2'},
                'Target': {'Index': 0, 'Processor': 'RNN Decoder Cell 3'}},
               {'Parent': 'h_i',
                'Source': {'Index': 1, 'Processor': 'RNN Decoder Cell 2'},
                'Target': {'Index': 1, 'Processor': 'RNN Decoder Cell 3'}},
                {'Parent': 'y_i',
                'Source': {'Index': 0, 'Processor': 'RNN Decoder Cell 1'},
                'Target': {'Index': 0, 'Processor': 'RNN Decoder Cell 2'}},
               {'Parent': 'h_i',
                'Source': {'Index': 1, 'Processor': 'RNN Decoder Cell 1'},
                'Target': {'Index': 1, 'Processor': 'RNN Decoder Cell 2'}}], auto_increment=True)

In [9]:
project.add_block(id="Output Layer Block Length 3",
                  name="Output Layer Block Length 3",
                  description="The output layer which puts together the output tokens into a vector",
                  domain=["y_i", "y_i", "y_i"],
                  codomain=["Y"],)


project.add_processor(id="Output Layer Length 3",
                  description="The output layer which puts together the output tokens into a vector",
                  parent_id="Output Layer Block Length 3",)

project.add_wires([{'Parent': 'y_i',
                'Source': {'Index': 0, 'Processor': 'RNN Decoder Cell 1'},
                'Target': {'Index': 0, 'Processor': "Output Layer Length 3"}},
                {'Parent': 'y_i',
                'Source': {'Index': 0, 'Processor': 'RNN Decoder Cell 2'},
                'Target': {'Index': 1, 'Processor': "Output Layer Length 3"}},
                {'Parent': 'y_i',
                'Source': {'Index': 0, 'Processor': 'RNN Decoder Cell 3'},
                'Target': {'Index': 2, 'Processor': "Output Layer Length 3"}}], auto_increment=True)

In [10]:
project.add_system(id="RNN Decoder System Length 3",
                   processors=['RNN Decoder Cell 1', 'RNN Decoder Cell 2', 'RNN Decoder Cell 3', 'Output Layer Length 3'],
                   wires=["W9", "W10", "W11", "W12", "W13", "W14", "W15"],
                     description="The RNN encoder system",)

project.systems_map["RNN Decoder System Length 3"].display_mermaid_graphic()

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GS0[RNN Decoder System Length 3]
subgraph G0[RNN Decoder Cell 1 - RNN Decoder Hidden Layer Block]
direction LR
X0[RNN Decoder Cell 1]
subgraph G0P[Ports]
direction TB
XX0P0[y_i]
XX0P1[h_i]
XX0P2[c]
end
XX0P0[y_i] o--o X0
XX0P1[h_i] o--o X0
XX0P2[c] o--o X0
subgraph G0T[Terminals]
direction TB
XX0T0[y_i]
XX0T1[h_i]
end
X0 o--o XX0T0[y_i]
X0 o--o XX0T1[h_i]
end
subgraph G1[RNN Decoder Cell 2 - RNN Decoder Hidden Layer Block]
direction LR
X1[RNN Decoder Cell 2]
subgraph G1P[Ports]
direction TB
XX1P0[y_i]
XX1P1[h_i]
XX1P2[c]
end
XX1P0[y_i] o--o X1
XX1P1[h_i] o--o X1
XX1P2[c] o--o X1
subgraph G1T[Terminals]
direction TB
XX1T0[y_i]
XX1T1[h_i]
end
X1 o--o XX1T0[y_i]
X1 o--o XX1T1[h_i]
end
subgraph G2[RNN Decoder Cell 3 - RNN Decoder Hidden Layer Block]
direction LR
X2[RNN Decoder Cell 3]
subgraph G2P[Ports]
direction TB
XX2P0[y_i]
XX2P1[h_i]
XX2P2[c]
end
XX2P0[y_i] o--o X2
XX2P1[h_i] o--o X2
XX2P2[c] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[y_i]
XX2T1[h_i]
end
X2 o--o XX2T0[y_i]
X2 o--o XX2T1[h_i]
end
subgraph G3[Output Layer Length 3 - Output Layer Block Length 3 Block]
direction LR
X3[Output Layer Length 3]
subgraph G3P[Ports]
direction TB
XX3P0[y_i]
XX3P1[y_i]
XX3P2[y_i]
end
XX3P0[y_i] o--o X3
XX3P1[y_i] o--o X3
XX3P2[y_i] o--o X3
subgraph G3T[Terminals]
direction TB
XX3T0[Y]
end
X3 o--o XX3T0[Y]
end
XX1T0[y_i] ---> XX2P0[y_i]
XX1T1[h_i] ---> XX2P1[h_i]
XX0T0[y_i] ---> XX1P0[y_i]
XX0T1[h_i] ---> XX1P1[h_i]
XX0T0[y_i] ---> XX3P0[y_i]
XX1T0[y_i] ---> XX3P1[y_i]
XX2T0[y_i] ---> XX3P2[y_i]
end

```

In [11]:
project.systems_map["RNN Decoder System Length 3"].make_processor_lazy(terminals=[["Output Layer Length 3", 0, "Y"]],
                                                                       block_id="RNN Decoder Block Length 3",
                                                                    block_name="RNN Decoder Block Length 3",
                                                                    processor_id="RNN Decoder Length 3",
                                                                    processor_name="RNN Decoder Length 3",
                                                                    block_description="An RNN Decoder Block with length 3",
                                                                    processor_description="An RNN Decoder with length 3",
                                                                    add=True,
                                                                    project=project,)

project.processors_map["RNN Decoder Length 3"].display_mermaid_graphic(composite=True)

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GC0[RNN Decoder Length 3 - RNN Decoder Block Length 3 Block]
direction LR
subgraph GS0[RNN Decoder System Length 3]
subgraph G1[RNN Decoder Cell 1 - RNN Decoder Hidden Layer Block]
direction LR
X1[RNN Decoder Cell 1]
subgraph G1P[Ports]
direction TB
XX1P0[y_i]
XX1P1[h_i]
XX1P2[c]
end
XX1P0[y_i] o--o X1
XX1P1[h_i] o--o X1
XX1P2[c] o--o X1
subgraph G1T[Terminals]
direction TB
XX1T0[y_i]
XX1T1[h_i]
end
X1 o--o XX1T0[y_i]
X1 o--o XX1T1[h_i]
end
subgraph G2[RNN Decoder Cell 2 - RNN Decoder Hidden Layer Block]
direction LR
X2[RNN Decoder Cell 2]
subgraph G2P[Ports]
direction TB
XX2P0[y_i]
XX2P1[h_i]
XX2P2[c]
end
XX2P0[y_i] o--o X2
XX2P1[h_i] o--o X2
XX2P2[c] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[y_i]
XX2T1[h_i]
end
X2 o--o XX2T0[y_i]
X2 o--o XX2T1[h_i]
end
subgraph G3[RNN Decoder Cell 3 - RNN Decoder Hidden Layer Block]
direction LR
X3[RNN Decoder Cell 3]
subgraph G3P[Ports]
direction TB
XX3P0[y_i]
XX3P1[h_i]
XX3P2[c]
end
XX3P0[y_i] o--o X3
XX3P1[h_i] o--o X3
XX3P2[c] o--o X3
subgraph G3T[Terminals]
direction TB
XX3T0[y_i]
XX3T1[h_i]
end
X3 o--o XX3T0[y_i]
X3 o--o XX3T1[h_i]
end
subgraph G4[Output Layer Length 3 - Output Layer Block Length 3 Block]
direction LR
X4[Output Layer Length 3]
subgraph G4P[Ports]
direction TB
XX4P0[y_i]
XX4P1[y_i]
XX4P2[y_i]
end
XX4P0[y_i] o--o X4
XX4P1[y_i] o--o X4
XX4P2[y_i] o--o X4
subgraph G4T[Terminals]
direction TB
XX4T0[Y]
end
X4 o--o XX4T0[Y]
end
XX2T0[y_i] ---> XX3P0[y_i]
XX2T1[h_i] ---> XX3P1[h_i]
XX1T0[y_i] ---> XX2P0[y_i]
XX1T1[h_i] ---> XX2P1[h_i]
XX1T0[y_i] ---> XX4P0[y_i]
XX2T0[y_i] ---> XX4P1[y_i]
XX3T0[y_i] ---> XX4P2[y_i]
end
subgraph GC0P[Ports]
direction TB
X1P0[y_i]
X1P1[h_i]
X1P2[c]
X1P3[c]
X1P4[c]
end
X1P0[y_i] --> XX1P0[y_i]
X1P1[h_i] --> XX1P1[h_i]
X1P2[c] --> XX1P2[c]
X1P3[c] --> XX2P2[c]
X1P4[c] --> XX3P2[c]
subgraph GC0T[Terminals]
direction TB
X1T0[Y]
end
XX4T0[Y] --> X1T0[Y]
end

```

In [12]:
project.processors_map["RNN Encoder Length 3"].find_potential_wires(project.processors_map["RNN Decoder Length 3"])

{'Ports': [],
 'Terminals': [{'Parent': 'c',
   'Source': {'Processor': 'RNN Encoder Length 3', 'Index': 0},
   'Target': {'Processor': 'RNN Decoder Length 3', 'Index': 2}},
  {'Parent': 'c',
   'Source': {'Processor': 'RNN Encoder Length 3', 'Index': 0},
   'Target': {'Processor': 'RNN Decoder Length 3', 'Index': 3}},
  {'Parent': 'c',
   'Source': {'Processor': 'RNN Encoder Length 3', 'Index': 0},
   'Target': {'Processor': 'RNN Decoder Length 3', 'Index': 4}}]}

In [13]:
project.add_wires([{'Parent': 'c',
   'Source': {'Processor': 'RNN Encoder Length 3', 'Index': 0},
   'Target': {'Processor': 'RNN Decoder Length 3', 'Index': 2}},
  {'Parent': 'c',
   'Source': {'Processor': 'RNN Encoder Length 3', 'Index': 0},
   'Target': {'Processor': 'RNN Decoder Length 3', 'Index': 3}},
  {'Parent': 'c',
   'Source': {'Processor': 'RNN Encoder Length 3', 'Index': 0},
   'Target': {'Processor': 'RNN Decoder Length 3', 'Index': 4}}], auto_increment=True)

In [14]:
project.add_system(id="RNN Encoder-Decoder System Length 3",
                   processors=["RNN Encoder Length 3", "RNN Decoder Length 3"],
                   wires=["W16", "W17", "W18"],
                     description="A length 3 RNN encoder-decoder system",)

project.systems_map["RNN Encoder-Decoder System Length 3"].display_mermaid_graphic()
project.systems_map["RNN Encoder-Decoder System Length 3"].display_mermaid_graphic(composite=True)

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GS0[RNN Encoder-Decoder System Length 3]
subgraph G0[RNN Encoder Length 3 - RNN Encoder Block Length 3 Block]
direction LR
X0[RNN Encoder Length 3]
subgraph G0P[Ports]
direction TB
XX0P0[h_i]
XX0P1[X]
end
XX0P0[h_i] o--o X0
XX0P1[X] o--o X0
subgraph G0T[Terminals]
direction TB
XX0T0[c]
end
X0 o--o XX0T0[c]
end
subgraph G1[RNN Decoder Length 3 - RNN Decoder Block Length 3 Block]
direction LR
X1[RNN Decoder Length 3]
subgraph G1P[Ports]
direction TB
XX1P0[y_i]
XX1P1[h_i]
XX1P2[c]
XX1P3[c]
XX1P4[c]
end
XX1P0[y_i] o--o X1
XX1P1[h_i] o--o X1
XX1P2[c] o--o X1
XX1P3[c] o--o X1
XX1P4[c] o--o X1
subgraph G1T[Terminals]
direction TB
XX1T0[Y]
end
X1 o--o XX1T0[Y]
end
XX0T0[c] ---> XX1P2[c]
XX0T0[c] ---> XX1P3[c]
XX0T0[c] ---> XX1P4[c]
end

```

```mermaid
---
config:
    layout: elk
---
graph LR
subgraph GS0[RNN Encoder-Decoder System Length 3]
subgraph GC0[RNN Encoder Length 3 - RNN Encoder Block Length 3 Block]
direction LR
subgraph GS1[RNN Encoder System Length 3]
subgraph G1[RNN Encoder Cell 1 - RNN Encoder Hidden Layer Block]
direction LR
X1[RNN Encoder Cell 1]
subgraph G1P[Ports]
direction TB
XX1P0[x_i]
XX1P1[h_i]
end
XX1P0[x_i] o--o X1
XX1P1[h_i] o--o X1
subgraph G1T[Terminals]
direction TB
XX1T0[h_i]
end
X1 o--o XX1T0[h_i]
end
subgraph G2[RNN Encoder Cell 2 - RNN Encoder Hidden Layer Block]
direction LR
X2[RNN Encoder Cell 2]
subgraph G2P[Ports]
direction TB
XX2P0[x_i]
XX2P1[h_i]
end
XX2P0[x_i] o--o X2
XX2P1[h_i] o--o X2
subgraph G2T[Terminals]
direction TB
XX2T0[h_i]
end
X2 o--o XX2T0[h_i]
end
subgraph G3[RNN Encoder Cell 3 - RNN Encoder Hidden Layer Block]
direction LR
X3[RNN Encoder Cell 3]
subgraph G3P[Ports]
direction TB
XX3P0[x_i]
XX3P1[h_i]
end
XX3P0[x_i] o--o X3
XX3P1[h_i] o--o X3
subgraph G3T[Terminals]
direction TB
XX3T0[h_i]
end
X3 o--o XX3T0[h_i]
end
subgraph G4[Input Layer Length 3 - Input Layer Block Length 3 Block]
direction LR
X4[Input Layer Length 3]
subgraph G4P[Ports]
direction TB
XX4P0[X]
end
XX4P0[X] o--o X4
subgraph G4T[Terminals]
direction TB
XX4T0[x_i]
XX4T1[x_i]
XX4T2[x_i]
end
X4 o--o XX4T0[x_i]
X4 o--o XX4T1[x_i]
X4 o--o XX4T2[x_i]
end
subgraph G5[Build Context Vector Length 3 - Build Context Vector Block Length 3 Block]
direction LR
X5[Build Context Vector Length 3]
subgraph G5P[Ports]
direction TB
XX5P0[h_i]
XX5P1[h_i]
XX5P2[h_i]
end
XX5P0[h_i] o--o X5
XX5P1[h_i] o--o X5
XX5P2[h_i] o--o X5
subgraph G5T[Terminals]
direction TB
XX5T0[c]
end
X5 o--o XX5T0[c]
end
XX1T0[h_i] ---> XX2P1[h_i]
XX2T0[h_i] ---> XX3P1[h_i]
XX4T0[x_i] ---> XX1P0[x_i]
XX4T1[x_i] ---> XX2P0[x_i]
XX4T2[x_i] ---> XX3P0[x_i]
XX1T0[h_i] ---> XX5P0[h_i]
XX2T0[h_i] ---> XX5P1[h_i]
XX3T0[h_i] ---> XX5P2[h_i]
end
subgraph GC0P[Ports]
direction TB
X2P0[h_i]
X2P1[X]
end
X2P0[h_i] --> XX1P1[h_i]
X2P1[X] --> XX4P0[X]
subgraph GC0T[Terminals]
direction TB
X2T0[c]
end
XX5T0[c] --> X2T0[c]
end
subgraph GC7[RNN Decoder Length 3 - RNN Decoder Block Length 3 Block]
direction LR
subgraph GS2[RNN Decoder System Length 3]
subgraph G8[RNN Decoder Cell 1 - RNN Decoder Hidden Layer Block]
direction LR
X8[RNN Decoder Cell 1]
subgraph G8P[Ports]
direction TB
XX8P0[y_i]
XX8P1[h_i]
XX8P2[c]
end
XX8P0[y_i] o--o X8
XX8P1[h_i] o--o X8
XX8P2[c] o--o X8
subgraph G8T[Terminals]
direction TB
XX8T0[y_i]
XX8T1[h_i]
end
X8 o--o XX8T0[y_i]
X8 o--o XX8T1[h_i]
end
subgraph G9[RNN Decoder Cell 2 - RNN Decoder Hidden Layer Block]
direction LR
X9[RNN Decoder Cell 2]
subgraph G9P[Ports]
direction TB
XX9P0[y_i]
XX9P1[h_i]
XX9P2[c]
end
XX9P0[y_i] o--o X9
XX9P1[h_i] o--o X9
XX9P2[c] o--o X9
subgraph G9T[Terminals]
direction TB
XX9T0[y_i]
XX9T1[h_i]
end
X9 o--o XX9T0[y_i]
X9 o--o XX9T1[h_i]
end
subgraph G10[RNN Decoder Cell 3 - RNN Decoder Hidden Layer Block]
direction LR
X10[RNN Decoder Cell 3]
subgraph G10P[Ports]
direction TB
XX10P0[y_i]
XX10P1[h_i]
XX10P2[c]
end
XX10P0[y_i] o--o X10
XX10P1[h_i] o--o X10
XX10P2[c] o--o X10
subgraph G10T[Terminals]
direction TB
XX10T0[y_i]
XX10T1[h_i]
end
X10 o--o XX10T0[y_i]
X10 o--o XX10T1[h_i]
end
subgraph G11[Output Layer Length 3 - Output Layer Block Length 3 Block]
direction LR
X11[Output Layer Length 3]
subgraph G11P[Ports]
direction TB
XX11P0[y_i]
XX11P1[y_i]
XX11P2[y_i]
end
XX11P0[y_i] o--o X11
XX11P1[y_i] o--o X11
XX11P2[y_i] o--o X11
subgraph G11T[Terminals]
direction TB
XX11T0[Y]
end
X11 o--o XX11T0[Y]
end
XX9T0[y_i] ---> XX10P0[y_i]
XX9T1[h_i] ---> XX10P1[h_i]
XX8T0[y_i] ---> XX9P0[y_i]
XX8T1[h_i] ---> XX9P1[h_i]
XX8T0[y_i] ---> XX11P0[y_i]
XX9T0[y_i] ---> XX11P1[y_i]
XX10T0[y_i] ---> XX11P2[y_i]
end
subgraph GC7P[Ports]
direction TB
X3P0[y_i]
X3P1[h_i]
X3P2[c]
X3P3[c]
X3P4[c]
end
X3P0[y_i] --> XX8P0[y_i]
X3P1[h_i] --> XX8P1[h_i]
X3P2[c] --> XX8P2[c]
X3P3[c] --> XX9P2[c]
X3P4[c] --> XX10P2[c]
subgraph GC7T[Terminals]
direction TB
X3T0[Y]
end
XX11T0[Y] --> X3T0[Y]
end
X2T0[c] ---> X3P2[c]
X2T0[c] ---> X3P3[c]
X2T0[c] ---> X3P4[c]
end

```